In [3]:
from cassandra.cluster import Cluster
import pandas as pd


In [4]:
cluster = Cluster(['127.0.0.1'])
session = cluster.connect('info_stock')

In [5]:
import re

# 🔍 List คำที่เกี่ยวข้องกับธุรกิจฮารอม
haram_keywords = [
    r'\bbank(s)?\b',
    r'\binsurance\b',
    r'\bcasino(s)?\b',
    r'\bgambling\b',
    r'\bporn(ography)?\b',
    r'\blottery\b',
    r'\bbrewery\b',
    r'\bliquor\b',
    r'\bpork\b',
    r'\bweapon(s)?\b',
    r'\btobacco\b',
    r'\bnight\s?club\b',
    r'\binterest\b',
    r'\bloan(s)?\b',
    r'\bmortgage\b',
    r'\binvestment\sbank\b'
]

def is_halal_by_regex(text):
    text = text.lower()

    for pattern in haram_keywords:
        if re.search(pattern, text):
            return False  # ❌ Haram
    return True  # ✅ Halal or Syubhah


In [23]:
rows = session.execute("""
    SELECT * FROM  financial_data ;
""")
df= pd.DataFrame(rows)
df.head()

,ticker,industry,interest_income,long_business_summary,name,sector,total_assets,total_debt,total_revenue
0,SIAM.BK,"Furnishings, Fixtures & Appliances",-0.02879,Siam Steel International Public Company Limite...,SIAM_SIAM STEEL INTERNATIONAL,Consumer Cyclical,1.0,2.040780e+08,1.773048e+09
1,KAMART.BK,Household & Personal Products,0.27538,"Karmarts Public Company Limited, together with...",KAMART_KARMARTS,Consumer Defensive,1.0,6.533160e+08,3.322751e+09
2,IFS.BK,Credit Services,0.40719,IFS Capital (Thailand) Public Company Limited ...,IFS_IFS CAPITAL (THAILAND),Financial Services,1.0,2.237478e+09,4.514385e+08
3,POPF.BK,,0.00000,Prime Office Leasehold Property Fund specializ...,POPF_PRIME OFFICE LEASEHOLD,,1.0,1.908280e+08,8.941708e+08
4,PYLON.BK,Engineering & Construction,0.08830,"Pylon Public Company Limited, together with it...",PYLON_PYLON,Industrials,1.0,6.043544e+07,9.071023e+08


In [24]:
df.shape

(921, 9)

In [17]:
df.columns

Index(['ticker', 'industry', 'interest_income', 'long_business_summary',
       'name', 'sector', 'total_assets', 'total_debt', 'total_revenue'],
      dtype='object')

In [26]:
rows = session.execute("SELECT ticker, name, sector, industry, long_business_summary FROM financial_data")

results = []

for row in rows:
    combined_text = f"{row.sector or ''} {row.industry or ''} {row.long_business_summary or ''}"
    halal = is_halal_by_regex(combined_text)

    results.append({
        'Ticker': row.ticker,
        'Name': row.name,
        'Sector': row.sector,
        'Industry': row.industry,
        'Halal': '✅' if halal else '❌'
    })

df = pd.DataFrame(results)
print(df.head())

      Ticker                           Name              Sector  \
0    SIAM.BK  SIAM_SIAM STEEL INTERNATIONAL   Consumer Cyclical   
1  KAMART.BK                KAMART_KARMARTS  Consumer Defensive   
2     IFS.BK     IFS_IFS CAPITAL (THAILAND)  Financial Services   
3    POPF.BK    POPF_PRIME OFFICE LEASEHOLD                       
4   PYLON.BK                    PYLON_PYLON         Industrials   

                             Industry Halal  
0  Furnishings, Fixtures & Appliances     ✅  
1       Household & Personal Products     ✅  
2                     Credit Services     ✅  
3                                         ✅  
4          Engineering & Construction     ✅  


In [27]:
# ✅ แสดงผลแบบตาราง
df = pd.DataFrame(results)
pd.set_option('display.max_colwidth', None)  # ให้ข้อความแสดงเต็ม
print(df.head(10))

      Ticker                             Name                  Sector  \
0    SIAM.BK    SIAM_SIAM STEEL INTERNATIONAL       Consumer Cyclical   
1  KAMART.BK                  KAMART_KARMARTS      Consumer Defensive   
2     IFS.BK       IFS_IFS CAPITAL (THAILAND)      Financial Services   
3    POPF.BK      POPF_PRIME OFFICE LEASEHOLD                           
4   PYLON.BK                      PYLON_PYLON             Industrials   
5    SYMC.BK      SYMC_SYMPHONY COMMUNICATION  Communication Services   
6   WHAIR.BK   WHAIR_WHA INDUSTRIAL LEASEHOLD             Real Estate   
7     THG.BK    THG_THONBURI HEALTHCARE GROUP              Healthcare   
8    DCON.BK               DCON_DCON PRODUCTS         Basic Materials   
9     TAN.BK  TAN_TANACHIRA RETAIL CORPORATIO       Consumer Cyclical   

                             Industry Halal  
0  Furnishings, Fixtures & Appliances     ✅  
1       Household & Personal Products     ✅  
2                     Credit Services     ✅  
3           